In [13]:
def factorial(x):
    """This is a recursive function
    to find the factorial of an integer"""
    print(x)
    if x == 1:
        return 1
    else:
        return (x * factorial(x-1))

factorial(3)


3
2
1


6

In [35]:
sum({2,2,2})

2

In [2]:
dict1 = {'abundance':60,
         'name':'c1',
         'child':[
                {
                'abundance':5,
                'name':'cc1',
                'child':
                    [
                        {
                        'abundance':2,
                        'name':'ccc1',
                        'child':[]
                        },
                        {
                        'abundance':2,
                        'name':'ccc1',
                        'child':[]
                        }
                    ]
                },
                {
                'name':'cc1',
                'abundance':6,
                'child':[]
                }
            ]
        }

def compute_abundance(d):
    sum_ab = sum([compute_abundance(c) for c in d['child']])
    print(d['name'],sum_ab)
    return d['abundance']

compute_abundance(dict1)


ccc1 0
ccc1 0
cc1 4
cc1 0
c1 11


60

In [16]:
import bz2
import pickle
from metaphlan.metaphlan import map2bbh
import numpy as np
import pandas as pd

with bz2.BZ2File( '/ssd1/wy/workspace/MetaPhlAn/test_data/databases/mpa_vJan21_TOY_CHOCOPhlAnSGB_202103.pkl', 'r' ) as a:
    mpa_pkl = pickle.load( a )
mpa = mpa_pkl
len(mpa['taxonomy'])
from metaphlan.metaphlan import TaxTree,map2bbh
tree = TaxTree( mpa_pkl, {} )

tree.set_min_cu_len( 2000 )



markers2reads, n_metagenome_reads, avg_read_length = map2bbh("/ssd1/wy/workspace/MetaPhlAn/test_data/01.fasta.bowtie2out.txt", 
                                                           5, "bowtie2out", None, None, None, '1992')
tree.set_stat( 'tavg_g',0.2, 0.33, avg_read_length,False)

map_out = []
for marker,reads in sorted(markers2reads.items(), key=lambda pars: pars[0]):
    if marker not in tree.markers2lens:
        continue
    tax_seq, ids_seq = tree.add_reads( marker, len(reads),
                                add_viruses = False,
                                ignore_eukaryotes = False,
                                ignore_bacteria = False,
                                ignore_archaea = False,
                                ignore_ksgbs = False,
                                ignore_usgbs = False
                                )
    if tax_seq:
        map_out +=["\t".join([r,tax_seq, ids_seq]) for r in sorted(reads)]

tax_lev =None
SGB_ANALYSIS = True
tax_units = "kpcofgst"

clade2abundance_n = dict([(tax_label, clade) for tax_label, clade in tree.all_clades.items()
                    if tax_label.startswith("k__") and not clade.uncl])

clade2abundance, clade2est_nreads, tot_ab, tot_reads = {}, {}, 0.0, 0



def compute_abundance( clade ):
    if clade.abundance is not None: return clade.abundance

    sum_ab = sum([compute_abundance(c) for c in clade.children.values()])


    rat_nreads, removed = [], []
    for marker, n_reads in sorted(clade.markers2nreads.items(),key=lambda x:x[0]):
        misidentified = False

        # if not clade.avoid_disqm:
        #     for ext in clade.markers2exts[marker]:
        #         ext_clade = clade.taxa2clades[ext]
        #         m2nr = ext_clade.markers2nreads
                
        #         tocladetmp = ext_clade
        #         while len(tocladetmp.children) == 1:
        #             tocladetmp = list(tocladetmp.children.values())[0]
        #             m2nr = tocladetmp.markers2nreads

        #         nonzeros = sum([v>0 for v in m2nr.values()])
        #         if len(m2nr):
        #             if float(nonzeros) / len(m2nr) > clade.perc_nonzero:
        #                 misidentified = True
        #                 removed.append( (clade.markers2lens[marker],n_reads) )
        #                 break
        if not misidentified:
            rat_nreads.append( (clade.markers2lens[marker],n_reads) )
        # if not clade.avoid_disqm and len(removed):
        #     n_rat_nreads = float(len(rat_nreads))
        #     n_removed = float(len(removed))
        #     n_tot = n_rat_nreads + n_removed
        #     n_ripr = 10

        #     if len(clade.get_terminals()) < 2:
        #         n_ripr = 0

        #     if "k__Viruses" in clade.get_full_name():
        #         n_ripr = 0

        #     if n_rat_nreads < n_ripr and n_tot > n_rat_nreads:
        #         rat_nreads += removed[:n_ripr-int(n_rat_nreads)]       
    rat_nreads = sorted(rat_nreads, key = lambda x: x[1])
    # rat_v： marker 基因的长度  nreads_v： 每一个marker基因对应的reads个数
    rat_v,nreads_v = zip(*rat_nreads) if rat_nreads else ([],[])
    # rat： marker 基因的长度总和 nrawreads：reads的个数
    rat, nrawreads, loc_ab = float(sum(rat_v)) or -1.0, sum(nreads_v), 0.0
    quant = int(clade.quantile*len(rat_nreads))
    ql,qr,qn = (quant,-quant,quant) if quant else (None,None,0)


    if rat < 0.0:
        pass
    elif clade.stat == 'tavg_g':
        # n: 每一个marker基因对应的reads个数; r: 每一个marker基因的长度; avg_read_length: 平均reads的长度
        wnreads = sorted([(float(n)/(np.absolute(r-clade.avg_read_length)+1),(np.absolute(r - clade.avg_read_length)+1) ,n) for r,n in rat_nreads], key=lambda x:x[0])
        den,num = zip(*[v[1:] for v in wnreads[ql:qr]])
        if sum(num) >0:
            print("111")
        loc_ab = float(sum(num))/float(sum(den)) if any(den) else 0.0

    clade.abundance = loc_ab
    if rat < clade.min_cu_len and clade.children:
        clade.abundance = sum_ab
    elif loc_ab < sum_ab:
        clade.abundance = sum_ab

    if clade.abundance > sum_ab and clade.children: # *1.1??
        clade.uncl_abundance = clade.abundance - sum_ab
    clade.subcl_uncl = not clade.children and clade.name[0] not in tax_units[-2:]

    return clade.abundance


for tax_label, clade in clade2abundance_n.items():
    tot_ab += compute_abundance(clade)

def get_all_abundances(clade ):

    ret = [(clade.name, clade.tax_id, clade.abundance)]
    if clade.uncl_abundance > 0.0:
        lchild = list(clade.children.values())[0].name[:3]
        ret += [(lchild+clade.name[3:]+"_unclassified", "", clade.uncl_abundance)]
    if clade.subcl_uncl and clade.name[0] != tax_units[-2]:
        cind = tax_units.index( clade.name[0] )
        ret += [(   tax_units[cind+1]+clade.name[1:]+"_unclassified","",
                    clade.abundance)]
    for c in clade.children.values():
        ret += get_all_abundances(c)
    return ret


for tax_label, clade in clade2abundance_n.items():
    # print(clade.get_all_abundances())

    # print(get_all_abundances(clade))

    for clade_label, tax_id, abundance in sorted(clade.get_all_abundances(), key=lambda pars:pars[0]):
       clade2abundance[(clade_label, tax_id)] = abundance

ret_d = dict([( tax, float(abundance) / tot_ab if tot_ab else 0.0) for tax, abundance in clade2abundance.items()])
outpred = [(taxstr, taxid,round(relab*100.0,5)) for (taxstr, taxid), relab in ret_d.items() if relab > 0.0]
for clade, taxid, relab in sorted(  outpred, reverse=True,
                                        key=lambda x:x[2]+(100.0*(8-(x[0].count("|"))))):
    add_repr = ''
    print( "\t".join( [clade, 
                                taxid, 
                                str(relab*1.0), 
                                add_repr
                            ] ) + "\n" )

In [ ]:
for marker,reads in sorted(markers2reads.items(), key=lambda pars: pars[0]):
    if marker not in tree.markers2lens:
        continue
    print(marker,reads)
    print(marker,len(reads))

In [8]:
for key in tree.all_clades:
    print(key,tree.all_clades[key].markers2nreads)

k__Bacteria {}
p__Bacteroidetes {}
c__Bacteroidia {}
o__Bacteroidales {}
f__Prevotellaceae {}
g__Prevotella {}
s__Prevotella_pallens {}
t__SGB1564 {'SGB1564__JGOLIIPB_00862': 0, 'SGB1564__GDOFNNPN_00972': 0, 'SGB1564__PMFBMJLI_00745': 0, 'SGB1564__NIOPKNIK_01028': 0, 'SGB1564__GFCLMLJA_00154': 0, 'SGB1564__CEIFGCFE_01072': 0, 'SGB1564__EGPHDCPP_00377': 0, 'SGB1564__OJBADEDG_00071': 0, 'SGB1564__BEJLHJHH_01229': 0, 'SGB1564__CJOOHOAN_01243': 0, 'SGB1564__PMFBMJLI_00153': 0, 'SGB1564__PDPKHGOH_00273': 0, 'SGB1564__GHJMIIGL_01772': 0, 'SGB1564__PONKLCOM_00669': 0, 'SGB1564__CDPDFBAO_00659': 0, 'SGB1564__CNACDFJM_01295': 0, 'SGB1564__NFCDOGNB_01998': 0, 'SGB1564__EDJDBMGL_00930': 0, 'SGB1564__JLENLNLF_01213': 0, 'SGB1564__JKODLDJD_00638': 0, 'SGB1564__CJIEKGGJ_00829': 0, 'SGB1564__NFCDOGNB_01632': 0, 'SGB1564__BFMHBOIG_00521': 0, 'SGB1564__EMIAAOBD_01724': 0, 'SGB1564__PONKLCOM_00848': 0, 'SGB1564__CEIFGCFE_00873': 0, 'SGB1564__JKODLDJD_00330': 0, 'SGB1564__LFMCKECB_00187': 0, 'SGB1564__CD

In [4]:
for clade, value in mpa['taxonomy'].items():
    print(clade,value)

k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_pallens|t__SGB1564 ('2|976|200643|171549|171552|838|60133|', 3107191)
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_histicola|t__SGB1543 ('2|976|200643|171549|171552|838|470565|', 3033525)
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidales_unclassified|g__Phocaeicola|s__Phocaeicola_vulgatus|t__SGB1814 ('2|976|200643|171549||909656|821|', 5224998)
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_caccae|t__SGB1877 ('2|976|200643|171549|815|816|47678|', 5218573)
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Tannerellaceae|g__Parabacteroides|s__Parabacteroides_distasonis|t__SGB1934 ('2|976|200643|171549|2005525|375288|823|', 5285465)
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipe

In [3]:
for k,p in mpa['markers'].items():
    print(k,p)

SGB1543__OFGJFGGB_00388 {'clade': 't__SGB1543', 'ext': [], 'len': 3174, 'taxon': 'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_histicola|t__SGB1543'}
SGB1543__JCHLCMHF_00138 {'clade': 't__SGB1543', 'ext': [], 'len': 2970, 'taxon': 'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_histicola|t__SGB1543'}
SGB1543__CDNDMBFI_01014 {'clade': 't__SGB1543', 'ext': [], 'len': 2736, 'taxon': 'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_histicola|t__SGB1543'}
SGB1543__AEEEPKBE_01007 {'clade': 't__SGB1543', 'ext': [], 'len': 2688, 'taxon': 'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_histicola|t__SGB1543'}
SGB1543__MFAIKBNE_01119 {'clade': 't__SGB1543', 'ext': [], 'len': 2601, 'taxon': 'k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevot